In [37]:
%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
from IPython.display import clear_output

import pandas as pd
import numpy as np

pd.options.display.max_rows = 500
pd.options.display.max_columns = 500
pd.options.display.width = 2000
pd.options.display.max_colwidth = 2000

display(HTML("<style>.container { width:75% !important; }</style>"))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# M1: raport badawczy

data: 13.02.2019

Zespół badawczy:
  - dr hab. inż. Mikołaj Morzy, prof. PP
  - inż Oliver Pieńkos

W ramach pierwszych trzech miesięcy pracy zespół badawczy zrealizował następujące zadania:

- przygotowanie schematu adnotacji faktur wchodzących w skład "złotego standardu"
- ręczna adnotacja 100 różnych faktur (nabywcy, sprzedawcy, faktury vat, faktury proforma, ...)
- przygotowanie początkowego zestawu filtrów do identyfikacji podstawowych encji występujących na fakturze (data wystawienia, data sprzedaży, numery NIP, numery kont bankowych, numery REGON)
- oprogramowanie systemu OCR (ang. *optical character recognition*) do automatycznego sczytania tekstu występującego na fakturze
- przetworzenie 100 faktur stanowiących "złoty standard" i zaaplikowanie filtrów do oznaczenia istotnych fragmentów tekstu stanowiącego wynik procesu sczytania faktury na podstawie zdjęcia wykonanego telefonem komórkowym
- wytreniowanie 4 modeli uczenia maszynowego (ang. *machine learning*) do automatycznej identyfikacji wybranych encji występujących na fakturze:

  - typ faktury
  - numer faktury
  - NIP sprzedawcy
  - kwota brutto

### Opis procesu adnotacji i czytania zdjęć faktur

Poniżej przedstawiamy przykład zastosowania opracowanych modeli do jednej faktury, a także pokazujemy podsumowanie wyniku uczenia dla faktur stanowiących "złoty standard".

Wyjściowy obraz faktury jest wynikiem fotografii wykonanej przy użyciu telefonu komórkowego. Należy tutaj podkreślić, że jest to obraz który powstał tylko w wersji cyfrowej fotografii, bez dostępu do treści oryginalnego dokumentu (innymi słowy, faktura istnieje tylko w wersji papierowej, nie mamy dostępu np. do dokumentu `*.pdf` który został wydrukowany. Stanowi to istotną różnicę w stosunku do systemów, które operują właśnie na dokumentach, które powstały w wyniku eksportu do formatu `*.pdf`, i w przypadku których ekstrakcja spójnego i poprawnego tekstu jest trywialna.

![nieprzetworzona faktura](m1/00c0c0fcfcfcfcfc.png)

Pierwszym krokiem było pzygotowanie modułu OCR i wykonanie początkowego sczytania tekstu. Niestety, próby przeprowadzone na trzech wybranych systemach (Google Tesseract, OCR East, Firebase OCR) nie przyniosły zadowalających rezultatów. Tekst na fakturze był czytany fragmentarycznie, co powodowało, że wyjściowy strumień tekstu stanowił chaotycznyh strumień (np. fragmenty numerów kont bankowych były losowo przemieszane).

Poniżej przedstawiamy wynik zaznaczenia fragmentów na fakturze przed przeprowadzeniem dodatkowych prac badawczych.

![przed grupowaniem](m1/beforeGrouping.png)

Jak widać, wiele fragmentów faktury jest rozpoznawanych przez dużą liczbę wzajemnie nakładających się prostokątów, co powoduje niemożność analizy tekstu stanowiącego wynik działania systemu OCR

Analiza tego stanu rzeczy przekonała nas, że konieczne jest przepisanie fragmentu systemu do rozpoznawania tekstu ze zdjęć w taki sposób, aby ściśle przylegające do siebie fragmenty łączyły się w jeden spójny obszar, dzięki czemu unikniemy dzielenia semantycznie jednorodnych fragmentów tekstu. Po przeprowadzeniu prac sposób identyfikacji tekstu na fakturach uległ znaczącej poprawie, poniżej przedstawiamy tę samą fakturę po przetworzeniu za pomocą poprawionego systemu OCR

![po poprawkach OCR](m1/after.png)

Jak widać, każda linia tekstu jest rozpoznawana w sposób ciągły. Poniżej tekstowa reprezentacja tej faktury w formie stanowiącej bezpośredni surowy wynik działania systemu OCR.

In [38]:
!cat m1/00c0c0fcfcfcfcfc.txt

—Sprzedawca— Strona 1 Kancelaria Doradcy Podatkowego Daniel Moskow Faktura VAT nr:| 62-010 STĘSZEWKO, KONWALIOWA 17| NIP : 956-203-90-29 Regon : 340122961 | 00032/2010| Bank : KREDYT BANK i | z dnia : - 02-05-2010| Konto : 47150010541210500730140000 Data sprzedaży: 02-05-2010 Oryginał —Nabywca— MATRIOSZKA MIKOŁAJ MORZY KIEKRZ, AKACJOWA 2A NIP : 7791957777 Forma płatności : PRZELEW 7 DNI Termin płatności : 09-05-2010 is Nazwa towaru / usługi I PkwU  [ llość [ Jm.  [|Cenajedn.netto| Wartość netto | VAT |1 USŁUGI ZGODNIE Z UMOWĄ (KSIĘGOWOŚĆ) "4000, // 20000 2000022) Wartość VAT Wartość brutto__ 20000] 4400] | 244,00)1 22% Razem | 20000] Bapłaty : 244,00 PLN s=wnie : dwieście czterdzieści cztery 0/100 PLN Doradcą /pódawkowyf wpisu 10613 podpis osoby upoważnionej do podpis osoby u powais osoby upoważnionei do odbioru faktury VAT wystawienia faktury VAT


Taki surowy strumień tekstu jest następnie przez nas poddawany filtrowaniu przez zbiór filtrów, które próbują zidentyfikować w tekście istotne elementy, takie jak:

- numer konta bankowego
- numer NIP
- numer REGON
- kod pocztowy
- fragmenty tekstu wyglądające jak daty
- fragmenty tekstu wyglądające jak kwoty pieniężne
- fragmenty tekstu sugerujące wartość stawki podatku

Poniżej przedstawiamy tę samą fakturę, po dodatkowym wzbogaceniu tekstu z OCR o znaczniki wskazujące na możliwe wystąpienia powyższych encji w tekście. Istotne jest, że nie polegamy na danych adnotowanych przez filtry, ponieważ filtry mają charakter regułowy (przykładowo, kod pocztowy jest rozpoznawany jako sekwencja `dd-ddd` gdzie `d` jest dowolną cyfrą, NIP jest rozpoznawany jako jeden ze wzorców `ddd-dd-dd-ddd`, `ddd-ddd-dd-dd`, lub `dddddddddd`, z ew. przedrostkiem reprezentującym kraj). Znaczniki wstawione w poniższym tekście stanowią jedynie wzbogacenie, wzmocnienie sygnału dla modułu uczenia maszynowego, za pomocą których model statystyczny szybciej uczy się rozpoznawać fragmenty tekstu reprezentujące te i inne encje.

In [39]:
!cat m1/00c0c0fcfcfcfcfc-annotated.txt

—Sprzedawca— Strona 1 Kancelaria Doradcy Podatkowego Daniel Moskow <INVOICE_TYPE>Faktura VAT</INVOICE_TYPE> nr:| <ZIP>62-010</ZIP> STĘSZEWKO, KONWALIOWA 17| NIP : <NIP>956-203-90-29</NIP> Regon : <REGON>340122961</REGON> | 00032/2010| Bank : KREDYT BANK i | z dnia : - <DATE>02-05-2010</DATE>| Konto : <BANK_NO>47150010541210500730140000</BANK_NO> Data sprzedaży: <DATE>02-05-2010</DATE> Oryginał —Nabywca— MATRIOSZKA MIKOŁAJ MORZY KIEKRZ, AKACJOWA 2A NIP : <NIP>7791957777</NIP> Forma płatności : PRZELEW 7 DNI Termin płatności : <DATE>09-05-2010</DATE> is Nazwa towaru / usługi I PkwU  [ llość [ Jm.  [|Cenajedn.netto| Wartość netto | VAT |1 USŁUGI ZGODNIE Z UMOWĄ (KSIĘGOWOŚĆ) "4000, // 20000 2000022) Wartość VAT Wartość brutto__ 20000] 4400] | <MONEY>244,00</MONEY>)1 <VAT_RATE>22%</VAT_RATE> Razem | 20000] Bapłaty : <MONEY>244,00</MONEY> PLN s=wnie : dwieście czterdzieści cztery 0/100 PLN Doradcą /pódawkowyf wpisu 10613 podpis osoby upoważnionej do podpis osoby u powais osoby upoważnionei d

### Opis procesu uczenia maszynowego

W dalszej części raportu prezentujemy fragmenty zbioru uczącego (ang. *training set*) składającego się ze 100 anotowanych i ręcznie opisanych faktur oraz prezentujemy uzyskane wyniki.

In [3]:
import spacy
import pandas as pd

df = pd.read_csv('../data/annotations/invoice_data.csv')

df['id'] = df.filename.apply(lambda x: x.split('.')[0])
df['ocr'] = df.id.apply(lambda x: ''.join(open(f'../data/raw_ocr/{x}.txt','r').readlines()))

Dla każdej faktury dokonujemy adnotacji następujących pól:

- typ faktury
- numer faktury
- numer konta sprzedawcy
- kwota brutto
- stawka VAT
- NIP sprzedawcy
- nazwa sprzedawcy

In [24]:
columns= ['typ_faktury','numer_faktury','numer_konta','stawka_VAT','sprzedawca_nip','kwota_brutto','sprzedawca','ocr']

df[columns][5:10]

,typ_faktury,numer_faktury,numer_konta,stawka_VAT,sprzedawca_nip,kwota_brutto,sprzedawca,ocr
5,fv,FV 3209/2018,04 1090 1463 0000 0001 2027 9571,23,7773004068,147.36,Firma Handlowo Usługowa Natalia Prymowicz,"/20183209/2EVVAT,/NeFAKTURA V.GoŚliTlinaMurowana GO1.201828.pia.daży:, sprzeda!oryginal - Idata 8[1.201828.""wystawienia:dataGoślinaMUTOWaNa GOŃNABYWCA:2-095 MFinans| RakowniaOWOC YZEDAWCA:zięby 4Biuro RGSPRZEDAWCĆ3 SzwarcUsługowadlowo UsRenataoweFirma Baa PrynowiczNatalia| Gośli!lina62-095 MMurowana GC1 63-174-30-00r„ieńska 4sito! 6„nieżnień-V-03vl.655-11-1ISESEu:NIP/I17-300-52-68NIEPKAR S2023 |957Bari |0001000 (46 (1090.jed. ||cena 3wartość |Kwartość ||proc|wartość |pnettojed. |||cena |netto|iedn. |(VAT|vaT| brutto |protto|siary| |Tość |(PKAIU)1 rowaru/usługiNazwa t|Lp-4,02)„80].113,4137,36PB» REZOLORIOWA PB| BENZYNA. BBRUTTOVATNETTO14.83147,36]00.„56]21,5113,8„60|urustu:Wartość UpFrzedplatplata:147,36]21,56]aplacono:119,1,80]| suxa|„00o zapłaty:Pozostalo do> do 2qocówkaolatności: 3Sposób pdn:z terminie: —vwag::147,3( ogóle:ertość 09' groszysześć q!!ieści :trzydzieś| złotych trsieden z)czterdzieśćzieści :Sto (Słownie:hana._perecz kjestemżesiadczan,QświaVATfaktury Vłwyst.yrawniony dt| dojestemżeu1adczam, żOśuadVATfaktury VP> odbioru £do 0niony 'rrauniof!0 Ybodl FulA J"
6,fv,F008030111800260316U,51 1320 2550 0290 0645 0000 7749,zw,5250007313,9.4,Poczta Polska S. A. Region Sieci Gen.Tadeusza Kościuszki 77,"22-11-20:| wystawienia: 22:Sprzedawca: MocDataS.A.Polski:: Poczta £n SieclRegio!1 Koścluszki 7:nTadeusza KGan.Poznań60-94ztowy Pły Poznań 1Urząd P| Pocztow! KościuszTadeusza-890 Po;' 5250007313""Pocztowy.Bank: IBank0645 |- rachunku: 511749-0290 |1320 22550 020000.51 1NrIR F008030111800260316UFAKTURA NNR| RENATA S:O RACHUNKOWO-FIN| SZWARC- FINANSOWE RENabywca: BBIURO |ZIĘBY «|-RAKOWNIAOŚLINA-R5 MUROWANA.-095 |62-0' 69917434143000NIP |sługi:-11-20wykonania u:/ towarów lilub |b zakończenia do.| dokonania lutstawy tlub :dostaData| 22-11-- Data z| zapłaty: 2|/karta p| Gotówka/ka| płatniczaForma 1| zapłaty: GWartośćWortośćKwotaStawkaWartośćCena jIlośćPKWIUjednusługiLp.Nazwa ttowaru /Jm.bruttoVAT)VATopustubrutto0,0080| USTVATA43U1P17 S:SZT|.OPŁATY 6,1ŻN.C„BO.I OWOCE K(KONWALIATY I„002,60| DSTVATA43U1P1I7 SZT„00350 |szrZNACZEK :3 GADARYRazemtym '0,009,40zwzapłać),40płacono:* ZŁOTYCH 4» DZIEWIĘĆ Z0/7100;łownie IA OLEJNICZAKDANUTA C(sobyIsko inazwis)fakturywysdymydoawnionej ćuprawni:Uwagi:"
7,fv,10107067/P203/2018,NaN,23,5862016763,145,JYSK Sp. z o. o.,"10107067/P203/2018FAKTURAFakturastawienia:27.08.2018wystData '>:000P203Sklep:iragonu:0734- pareNr pKasa:101Klient:JYSK |Sp.NOSOSrzedawca: JRENATA S-FINANSOWE RE5 RACHUNKOWO-FIBIURO |,/RAKOWNIA4 GOSLINA/RA(| MUROWANA GX-095Adres:80-299 GGdańskres:ZIĘBY 4Meteorytoytowa |]6991743000NIP:5 20 1!16 74| 000002848BDOlartośćKwotaWartość KVATKwoteIlośćCenaNEJed-NumerbruttoVATRabatubruttonettostkaPKWIU rnostlartykułuazwa .miary145,00ZL|,00117,89299,00SZTKRZ KZZ KASTRUP s.> kolog51.51 %3644350Suma145,00EM117,38Kwota23%0,000,0000KwotaMUJ0,00„00Kwota145,00117,8SumaPLN5,00 E'erminalZapłaconoPLN145,00 ISumasprzedawcyPodpisklientaPodpis +000| POZYCJAPOZYCJAKSIĘGIREJESTRU VAVAT"
8,fv,36368/SYS/2018,43 1160 2244 7406 0000 0000 6008,23,7773030567,71.3,Systemia.pl Sp. z o. o.,"Sssyustemia ralr36368/S$YS/2018tła wystawienia: 201: 2018/11/01Faktura V,Data vIVATRNr :| płatności: 2!Termin pORYGINAŁ2018/11/15| uowoczeEswość w uW MULTIMEDIACH) płatności: PrTyPI: PrzelewSprzedawca:Systemia.pl Sl Sp.z0.0.|. Błażeja 61| 6f/45.ul. I8 Poznań51-608 F303-05-67NIP 777RO RACHUNKOWO-BIURORRachun ek ban] 0000 63 0000 01160 z6008| 74060K bankowy: 43'43) 22447A SZWARC' RENATA SZFINANSOWE REa 25b/25Karpia 2Nabywca:61-620 F) PoznańBIURO R/4 SZWARC._ RENATA SZACHUNKOWO-FINANSOWE RENAul. 2. Zięby 462-095 F; Rakownia> 699 1NIP 6) 004 30 0!6008klienta: 61Numer kiWartośćKwotaWartość || ość: |Cena„PTU: || Rabat: || JM: |StP2 produktu, tousługi:Mi towaru hNazwa pbrutto: |netto:Jedn.netto:23%VAT 254.00 zijs 2018/11/01 -| ""Dobra C|

W kolejnym kroku tworzymy definicję modelu uczenia maszynowego, które na podstawie tekstu z systemu OCR przewiduje wartość atrybutu `typ_faktury`

In [34]:
df[['ocr','numer_konta']].to_csv('train.csv', header=['doc_txt','class'], index=False)

In [12]:
%%writefile model_definition.yaml

input_features:
    -
        name: doc_txt
        type: text
        encoder: cnnrnn
        cell_type: lstm
        level: char

output_features:
    -
        name: class
        type: category
            
training:
    learning_rate: 0.001
    early_stop: 10
    batch_size: 5
    learning_rate_warmup_epochs: 3
    validation_measure: accuracy


Overwriting model_definition.yaml


In [13]:
!ludwig train --data_csv train.csv --model_definition_file model_definition.yaml > m1/typ_faktury_train.txt

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

/home/mikolaj/.virtualenvs/python3/lib/python3.7/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)
Instructions for updating:
Use keras.layers.MaxPooling1D instead.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equiva

In [15]:
!cat m1/typ_faktury_train.txt

███████████████████████
█ █ █ █  ▜█ █ █ █ █   █
█ █ █ █ █ █ █ █ █ █ ███
█ █   █ █ █ █ █ █ █ ▌ █
█ █████ █ █ █ █ █ █ █ █
█     █  ▟█     █ █   █
███████████████████████
ludwig v0.2.1 - Train

Experiment name: experiment
Model name: run
Output path: results/experiment_run_0


ludwig_version: '0.2.1'
command: ('/home/mikolaj/.virtualenvs/python3/bin/ludwig train --data_csv train.csv '
 '--model_definition_file model_definition.yaml')
commit_hash: 'a51d1d60995e'
random_seed: 42
input_data: 'train.csv'
model_definition: {   'combiner': {'type': 'concat'},
    'input_features': [   {   'cell_type': 'lstm',
                              'encoder': 'cnnrnn',
                              'level': 'char',
                              'name': 'doc_txt',
                              'tied_weights': None,
                              'type': 'text'}],
    'output_features': [   {   'dependencies': [],
                               'loss': {   'class_similarities_temperature': 0,
              

Powtórzyliśmy proces uczenia maszynowego w celu wytreniowania pozostałych modeli, w celu oszczędzenia miejsca raportujemy wyniki ostatniej epoki uczenia dla rozpoznawania kolejnych encji.

#### NIP sprzedawcy

In [31]:
!cat m1/sprzedawca_nip_train.txt

╒═════════╤════════╤════════════╤═════════════╕
│ class   │   loss │   accuracy │   hits_at_k │
╞═════════╪════════╪════════════╪═════════════╡
│ train   │ 0.6995 │     0.8000 │      0.8571 │
├─────────┼────────┼────────────┼─────────────┤
│ vali    │ 5.4351 │     0.2500 │      0.3333 │
├─────────┼────────┼────────────┼─────────────┤
│ test    │ 6.6674 │     0.0556 │      0.0556 │
╘═════════╧════════╧════════════╧═════════════╛
Last improvement of accuracy on combined happened 10 epochs ago

EARLY STOPPING due to lack of validation improvement, it has been 10 epochs since last validation accuracy improvement

Best validation model epoch: 4
Best validation model accuracy on validation set combined: 0.25
Best validation model accuracy on test set combined: 0.1111111111111111

Finished: experiment_run
Saved to: results/experiment_run


#### Kwota brutto

In [32]:
!cat m1/kwota_brutto_train.txt

╒═════════╤════════╤════════════╤═════════════╕
│ class   │   loss │   accuracy │   hits_at_k │
╞═════════╪════════╪════════════╪═════════════╡
│ train   │ 0.8130 │     0.7571 │      0.8000 │
├─────────┼────────┼────────────┼─────────────┤
│ vali    │ 7.1464 │     0.0000 │      0.0000 │
├─────────┼────────┼────────────┼─────────────┤
│ test    │ 6.8878 │     0.0000 │      0.0000 │
╘═════════╧════════╧════════════╧═════════════╛
Last improvement of accuracy on combined happened 10 epochs ago

EARLY STOPPING due to lack of validation improvement, it has been 10 epochs since last validation accuracy improvement

Best validation model epoch: 1
Best validation model accuracy on validation set combined: 0.0
Best validation model accuracy on test set combined: 0.0

Finished: experiment_run
Saved to: results/experiment_run


#### Numer faktury

In [33]:
!cat m1/numer_faktury_train.txt

╒═════════╤════════╤════════════╤═════════════╕
│ class   │   loss │   accuracy │   hits_at_k │
╞═════════╪════════╪════════════╪═════════════╡
│ train   │ 0.6528 │     0.7857 │      0.8286 │
├─────────┼────────┼────────────┼─────────────┤
│ vali    │ 7.5175 │     0.0000 │      0.0000 │
├─────────┼────────┼────────────┼─────────────┤
│ test    │ 7.5949 │     0.0000 │      0.0000 │
╘═════════╧════════╧════════════╧═════════════╛
Last improvement of accuracy on combined happened 10 epochs ago

EARLY STOPPING due to lack of validation improvement, it has been 10 epochs since last validation accuracy improvement

Best validation model epoch: 1
Best validation model accuracy on validation set combined: 0.0
Best validation model accuracy on test set combined: 0.0

Finished: experiment_run
Saved to: results/experiment_run


#### Numer konta sprzedawcy

In [36]:
!cat m1/numer_konta_train.txt

╒═════════╤════════╤════════════╤═════════════╕
│ class   │   loss │   accuracy │   hits_at_k │
╞═════════╪════════╪════════════╪═════════════╡
│ train   │ 0.5323 │     0.8143 │      0.8857 │
├─────────┼────────┼────────────┼─────────────┤
│ vali    │ 7.1455 │     0.2500 │      0.2500 │
├─────────┼────────┼────────────┼─────────────┤
│ test    │ 6.7098 │     0.1111 │      0.3333 │
╘═════════╧════════╧════════════╧═════════════╛
Last improvement of accuracy on combined happened 10 epochs ago

EARLY STOPPING due to lack of validation improvement, it has been 10 epochs since last validation accuracy improvement

Best validation model epoch: 4
Best validation model accuracy on validation set combined: 0.25
Best validation model accuracy on test set combined: 0.16666666666666666

Finished: experiment_run
Saved to: results/experiment_run


## Podsumowanie

Zadania badawcze zdefiniowane w ramach kamienia milowego M1 zostały zrealizowane. Dokonano ręcznej adnotacji 100 faktur i uruchomiono pierwszy zestaw modeli uczenia maszynowego wytrenowanych na tym zbiorze. Dla wybranych encji uzyskano następującą dokładność:

- typ faktury: 77%
- NIP sprzedawcy: 80%
- kwota brutto: 76%
- numer faktury: 79%
- numer konta: 81%

Oczywiście należy podkreślić, że są to wyniki wstępne. Dane wejściowe zostaną poddane jeszcze dokładniejszej adnotacji, dodamy także więcej filtrów identyfikujących ważne elementy faktury, modele ML staną się też bardziej stabilne wraz ze wzrostem liczby przetworzonych faktur.

W najbliższym czasie planujemy realizację następujących kroków:

- dodanie filtrów rozpoznających:
  - kwoty wpisane ręcznie
  - nazwy ulic i miast
  - liczby sztuk
  - numery NIP spoza Polski
  - faktury zagraniczne
- poprawienie systemu OCR w taki sposób, aby łączyć prostokąty opakowujące (ang. *bounding box*) także w pionie, w celu identyfikacji danych wpisanych do faktury w formacie tabelarycznym
- rozszerzenie informacji ręcznie adnotowanych w "złotym standardzie" o wszystkie cechy faktury zdefiniowane w schemacie adnotacji
- zasilenie zbioru uczącego fakturami spoza "złotego standardu" (czyli fakturami nieposiadającymi ręcznej adnotacji) i przetestowanie precyzji modeli na takich fakturach
- rozpoczęcie eksperymentów z paragonami